In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

import datetime
from cityiq import Config, CityIq
from cityiq.api import Asset, Location
from cityiq.scrape import EventScraper

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

In [15]:
tracts = pkg.reference('tract_boundaries').geoframe()
comm = pkg.reference('sd_community_boundaries').geoframe()


{'init': 'epsg:4326'}

In [4]:

tz = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo

config = Config()
ciq = CityIq(config, start_time='20180901')

In [5]:
roads = pkg.reference('sdroads').geoframe()
sdroads = roads[(roads.lpsjur == 'SD') | (roads.rpsjur == 'SD')].copy()

# Convert to EPSG:2875 ( California zone 6, feet ) so we can make the buffer in feet
sdroads = sdroads.to_crs({'init':'epsg:2875'})

# Buffer the roads to be big enough to enclose the assets
sdroads['geometry'] = sdroads.geometry.apply(lambda g : g.buffer(60,cap_style=1, join_style=2))

sdroads = sdroads.to_crs( {'init':'epsg:4326'} )
# sdroads.to_csv('sdroads.csv')

In [28]:
assets = gpd.GeoDataFrame(ciq.asset_dataframe, geometry='geometry')
l1 = len(assets)
print(len(assets))
assets.crs = {'init':'epsg:4326'} 

ac = assets.copy()
ac['geometry'] = ac.centroid.to_crs({'init':'epsg:2875'}).buffer(20).to_crs( {'init':'epsg:4326'} )

# Link to roads
t = gpd.sjoin(ac, sdroads, how='left').drop(columns=['index_right'])

# Link to communities
t = gpd.sjoin(t, comm[comm['type'] == 'sd_community'][['geometry','name']], how='left').drop(columns=['index_right'])

# Link to tracts
t = gpd.sjoin(t, tracts[['geometry','geoid']], how='left')

t = t[list(assets)[:-1] + 
           ['name','geoid','roadsegid', 'speed', 'oneway','abloaddr', 'abhiaddr', 'rd30full'] +
           ['geometry'] ].drop_duplicates(subset='assetUid')\
           .rename(columns={'name':'community_name','geoid':'tract_geoid'})

assets = t.drop(columns='geometry').join(assets['geometry'])

assert len(assets) == l1
           
# How many assets and locations did not get linked to roads?
len(assets[assets.speed.isnull()]), len(assets)
           

26774


(54, 26774)

In [36]:
locations = gpd.GeoDataFrame(ciq.locations_dataframe, geometry='geometry')

l1 = len(locations)
locations.crs = {'init':'epsg:4326'}
lc = locations.copy()

lc['geometry'] =  locations.centroid.to_crs({'init':'epsg:2875'}).buffer(20).to_crs( {'init':'epsg:4326'} )

# Link to roads
t = gpd.sjoin(lc, sdroads, how='left').drop(columns=['index_right'])

# Link to communities
t = gpd.sjoin(t, comm[comm['type'] == 'sd_community'][['geometry','name']], how='left').drop(columns=['index_right'])

# Link to tracts
t = gpd.sjoin(t, tracts[['geometry','geoid']], how='left')

t = t[list(lc)[:-1] + 
           ['name','geoid','roadsegid', 'speed', 'oneway','abloaddr', 'abhiaddr', 'rd30full'] +
           ['geometry'] ].drop_duplicates(subset='locationUid')\
           .rename(columns={'name':'community_name','geoid':'tract_geoid'})

locations = t.drop(columns='geometry').join(locations['geometry'])


assert len(locations) == l1

In [38]:
locations.head()

,locationUid,locationType,parentLocationUid,community_name,tract_geoid,roadsegid,speed,oneway,abloaddr,abhiaddr,rd30full,geometry
0,004361eb,WALKWAY,004361eb,Downtown,14000US00000006974,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...
1,00456472,TRAFFIC_LANE,00456472,Downtown,14000US00000006974,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1705245131968 32.726558517207...
2,0051796c,WALKWAY,0051796c,Downtown,14000US00000006974,54946.0,20.0,F,1600.0,1699.0,STATE ST,LINESTRING (-117.1666581391174 32.722214661862...
3,00537bf3,TRAFFIC_LANE,00537bf3,Downtown,14000US00000007137,54946.0,20.0,F,1600.0,1699.0,STATE ST,LINESTRING (-117.166581746989 32.7222178855665...
4,005f90ed,WALKWAY,005f90ed,Downtown,14000US00000006974,54945.0,20.0,F,1700.0,1799.0,STATE ST,LINESTRING (-117.1665182431566 32.723810889902...
